In [2]:
import os
current_dir = os.getcwd()  #Here
# List the contents of the "data" directory
train_path = os.path.join(current_dir, "training_images")
data_directory_contents = os.listdir(train_path)
data_directory_contents[:5]

['vid_4_18340.jpg',
 'vid_4_2460.jpg',
 'vid_4_10720.jpg',
 'vid_4_14220.jpg',
 'vid_4_6160.jpg']

In [3]:
current_dir

'/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection'

In [4]:
import os
current_dir = os.getcwd()  #Here
# List the contents of the "data" directory
test_path = os.path.join(current_dir, "testing_images")
data_directory_contents = os.listdir(test_path)
data_directory_contents[:5]

['vid_5_30040.jpg',
 'vid_5_27500.jpg',
 'vid_5_28380.jpg',
 'vid_5_26420.jpg',
 'vid_5_31160.jpg']

In [5]:
import pandas as pd

# Load the bounding boxes CSV file
bounding_boxes_csv_path = os.path.join(current_dir, 'train_solution_bounding_boxes.csv')
bounding_boxes_df = pd.read_csv(bounding_boxes_csv_path)

# Display the first few rows of the CSV file
bounding_boxes_df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


# To YOLO Format

In [6]:
from PIL import Image
import os

def convert_to_yolo_format(row, img_folder_path):
    """
    Convert the bounding box format to YOLO format.
    YOLO format: <class> <x_center> <y_center> <width> <height>
    All the values are normalized between 0 and 1 with respect to the image's width and height.
    """
    # Open the image and get its size
    img_path = os.path.join(img_folder_path, row['image'])
    with Image.open(img_path) as img:
        img_width, img_height = img.size

    # Calculate bounding box parameters
    x_center = (row['xmin'] + row['xmax']) / 2.0
    y_center = (row['ymin'] + row['ymax']) / 2.0
    width = row['xmax'] - row['xmin']
    height = row['ymax'] - row['ymin']

    # Normalize the values
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height

    # Class index (assuming a single class dataset)
    class_index = 0

    return f"{class_index} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

# Test the function on the first row of the dataframe
yolo_format_example = convert_to_yolo_format(bounding_boxes_df.iloc[0], train_path)
yolo_format_example

'0 0.450434 0.539817 0.068741 0.095238'

In [7]:
# Create YOLO formatted annotations and save them as .txt files
output_folder = 'yolo3_annotations'
output_folder = os.path.join(current_dir,output_folder)
# Ensure the output directory exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for _, row in bounding_boxes_df.iterrows():
    yolo_data = convert_to_yolo_format(row,train_path)
    
    # Generate output file path
    output_file_name = os.path.splitext(row['image'])[0] + '.txt'
    output_path = os.path.join(output_folder, output_file_name)
    
    # Write to the file
    with open(output_path, 'a') as file:
        file.write(yolo_data + '\n')

# Check the first few files in the output directory
yolo_annotations_files = os.listdir(output_folder)[:5]
yolo_annotations_files

['vid_4_740.txt',
 'vid_4_6260.txt',
 'vid_4_13840.txt',
 'vid_4_15000.txt',
 'vid_4_21400.txt']

In [8]:
classes_path = os.path.join(current_dir, "classes.names")

# Create classes.names file
with open(classes_path, 'w') as file:
    file.write('car\n')  # Assuming "자동차" is the class name for cars

In [9]:
import random

# List all images in the training directory
all_images = [f for f in os.listdir(train_path) if f.endswith('.jpg')]
random.shuffle(all_images)

# Split the images into train and validation sets (80-20 split)
split_index = int(0.8 * len(all_images))
train_images = all_images[:split_index]
valid_images = all_images[split_index:]

# Save the list of train and validation images to train.txt and valid.txt
with open(os.path.join(current_dir, "yolo3_annotations", "train.txt"), 'w') as file:
    for img in train_images:
        file.write(os.path.join("training_images", img) + '\n')

with open(os.path.join(current_dir, "yolo3_annotations", "valid.txt"), 'w') as file:
    for img in valid_images:
        file.write(os.path.join("training_images", img) + '\n')

num_train_images = len(train_images)
num_valid_images = len(valid_images)

In [10]:
# Creating the yolov3-custom.cfg content
cfg_content = """
[net]
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 2000
policy=steps
steps=1600,1800
scales=.1,.1

# ...

[convolutional]
size=1
stride=1
pad=1
filters=18
activation=linear

[yolo]
mask = 0,1,2
anchors = 10,13, 16,30, 33,23, 30,61, 62,45, 59,119, 116,90, 156,198, 373,326
classes=1
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
"""

# Saving the cfg content to a file
cfg_path = os.path.join(current_dir, "yolov3-custom.cfg")
with open(cfg_path, 'w') as cfg_file:
    cfg_file.write(cfg_content)

cfg_path

'/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolov3-custom.cfg'

In [11]:

# Creating the data.yaml content
data_yaml_content = """
train: yolo3_annotations/train.txt
val: yolo3_annotations/valid.txt

nc: 1
names: ['car']
"""

# Saving the yaml content to a file
data_yaml_path = os.path.join(current_dir, "yolo3_annotations", "data.yaml")
with open(data_yaml_path, 'w') as yaml_file:
    yaml_file.write(data_yaml_content)

data_yaml_path

'/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolo3_annotations/data.yaml'

In [14]:
MY_PATH="/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolov3"
CFG_PATH="/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolov3-custom.cfg"
DATA_PATH="/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolo3_annotations/data.yaml"
! python $MY_PATH/train.py --img 416 --batch 16 --epochs 1 --data $DATA_PATH --cfg $CFG_PATH --weights yolov3.pt


train: weights=yolov3.pt, cfg=/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolov3-custom.cfg, data=/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolo3_annotations/data.yaml, hyp=yolov3/data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov3/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 15921, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 15921 (delta 17), reused 28 (delta 13), pack-reused 15880
Receiving objects: 100% (15921/15921), 14.66 MiB | 